# Conn Settings

In [26]:
import sqlite3
import pandas as pd
import warnings
import numpy as np

warnings.filterwarnings('ignore')

In [27]:
conn = sqlite3.connect("main.db")

# Import To Budget_Rad

In [28]:
raw = pd.read_csv(r"C:\Users\adamss\OneDrive - Central Health\Budget Project\csvUploads\budget.csv")

In [29]:
raw.head()

,Budget Id,Business Unit Id,Chart Id,Account No.,Amount,Currency Id,Base Amount,Business Unit,Chart,Account,...,ExchRate2,TranslatedAmt3,ExchRate3,TranslatedAmt4,ExchRate4,TranslatedAmt5,ExchRate5,Calc1,Calc2,Calc3
0,FY21,4999,1,50000,0,USD,0,HCD - DNU Archive - S&R Health,Chart of Account,Salaries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FY21,4999,1,50000,0,USD,0,HCD - DNU Archive - S&R Health,Chart of Account,Salaries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FY21,4999,1,50000,0,USD,0,HCD - DNU Archive - S&R Health,Chart of Account,Salaries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FY21,4999,1,50000,0,USD,0,HCD - DNU Archive - S&R Health,Chart of Account,Salaries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FY21,9550,1,50000,"5,291.73",USD,"5,291.73",Procurement,Chart of Account,Salaries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df = raw.copy()
df = df.reset_index(drop=True)
df['Id'] = df.index + 1
df.head(2)

,Budget Id,Business Unit Id,Chart Id,Account No.,Amount,Currency Id,Base Amount,Business Unit,Chart,Account,...,TranslatedAmt3,ExchRate3,TranslatedAmt4,ExchRate4,TranslatedAmt5,ExchRate5,Calc1,Calc2,Calc3,Id
0,FY21,4999,1,50000,0,USD,0,HCD - DNU Archive - S&R Health,Chart of Account,Salaries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,FY21,4999,1,50000,0,USD,0,HCD - DNU Archive - S&R Health,Chart of Account,Salaries,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [31]:
df_split = df.copy()
cols = [col for col in df_split.columns if "RAD" in col and "Description" not in col]

In [32]:

df_split[cols] = df_split[cols].applymap(lambda x: str(x) if not pd.isna(x) else np.nan)
df_split["Combined"] = df_split[cols].apply(
    lambda row: "|".join([str(x) for x in row if not pd.isna(x)]), axis=1
)
df_split = df_split.assign(value=df_split['Combined'].str.split('|')).explode('value')
df_result = df_split[['Id', 'value']].reset_index(drop=True)

In [33]:
df_result.replace("", float("NaN"), inplace=True)
df_result.dropna(inplace=True)
df_result.rename(columns={'value': 'RADId', 'Id': 'BudgetId'}, inplace=True)
df_result

,BudgetId,RADId
0,1,OVERTIME
1,2,OVERTIME
2,3,OVERTIME
3,4,OVERTIME
4,5,OVERTIME
...,...,...
34301,32358,DURABLE MED
34390,32447,PULMONOLOGY
34391,32448,CARDIOLOGY
34412,32469,CHIF


In [37]:
def import_func(df: pd.DataFrame()) -> None:
    df.to_sql('Budget_Rad', conn, if_exists='append', index=False)
    
import_func(df_result)

# Import To Budget

In [35]:
df = raw.copy()
df.columns = df.columns.str.replace(" ", "")
df = df.reset_index(drop=True)
df["Id"] = df.index + 1
df = df.rename(columns={'BudgetId': 'FiscalYear', 'AccountNo.': 'AccountNo'})
df = df[["Id", "FiscalYear", "BusinessUnitId", "AccountNo", "Amount"]]

In [36]:
def import_func(df: pd.DataFrame()) -> None:
    df.to_sql('Budget', conn, if_exists='append', index=False)
    
# import_func(df)